# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
cities = pd.read_csv("output_data/cities.csv")
cities

,City,Country,Date,Lat,Lng,Max Temp,Wind Speed,Cloudiness,Humidity
0,Hermanus,ZA,1592405778,-34.42,19.23,294.82,1.46,0,49
1,Ponta do Sol,PT,1592405790,32.67,-17.10,296.15,2.10,40,78
2,Mar del Plata,AR,1592405540,-38.00,-57.56,290.15,2.60,75,93
3,Katsuura,JP,1592405889,35.13,140.30,295.15,2.10,75,83
4,Manono,CD,1592406345,-7.30,27.42,303.39,2.97,0,24
...,...,...,...,...,...,...,...,...,...
578,Vega de Alatorre,MX,1592406508,20.03,-96.63,301.30,2.22,0,67
579,Dekar,BW,1592406508,-21.53,21.93,293.74,4.35,0,31
580,Verkhoyansk,RU,1592406508,67.54,133.38,291.81,2.95,48,49
581,Catalão,BR,1592406508,-18.17,-47.95,296.23,6.27,0,56


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [7]:
# Configure gmaps
gmaps.configure(api_key=g_key)
m=gmaps.Map()
m.add_layer(gmaps.WeightedHeatmap(data=cities))

TraitError: The 'locations' trait of a WeightedHeatmap instance must be of length 1 <= L <= 9223372036854775807, but a value of [] was specified.

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display Map